# 04_demand_typology

In [ ]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Load Data
df_all = pd.concat([pd.read_csv(f) for f in ['district_monthly_enrollment.csv', 'district_monthly_biometric.csv', 'district_monthly_demographic.csv']])
# Need Age columns summed
age_sum = df_all.groupby(['state', 'district'])[['age_0_5', 'age_5_17', 'age_18_plus', 'total']].sum().reset_index()

# Join with Pressure Tier from 03
pressure = pd.read_csv('03_output.csv')
combined = age_sum.merge(pressure[['state', 'district', 'pressure_tier', 'volatility']], on=['state', 'district'])
# Join with Dominance from 02 (for Biometric Dominant check)
dom = pd.read_csv('02_output.csv')
combined = combined.merge(dom[['state', 'district', 'dominant_type']], on=['state', 'district'])


In [ ]:
# Compute Ratios
combined['child_pressure'] = combined['age_0_5'] + combined['age_5_17']
combined['adult_pressure'] = combined['age_18_plus']
combined['child_ratio'] = combined['child_pressure'] / combined['total']
combined['adult_ratio'] = combined['adult_pressure'] / combined['total']

# Classification
def get_typology(row):
    tier = row['pressure_tier']
    c_ratio = row['child_ratio']
    a_ratio = row['adult_ratio']
    is_high_crit = tier in ['High Stress', 'Critical Infrastructure Stress']
    
    if c_ratio > 0.6 and is_high_crit:
        return 'School-linked Surge Zone'
    if a_ratio > 0.7 and is_high_crit:
        return 'Correctional Overload Zone'
    if row['dominant_type'] == 'Biometric' and row['volatility'] > 100: # Heuristic for High Volatility
        return 'Migration Impact Zone'
    return 'Population Expansion Zone'

combined['typology'] = combined.apply(get_typology, axis=1)

# Output
out_04 = combined[['state', 'district', 'typology', 'child_ratio', 'adult_ratio', 'pressure_tier']]
out_04.round(3).to_csv('04_output.csv', index=False)
print('Saved 04_output.csv')